In [1]:
import requests
import pandas as pd
import numpy as np
import requests 
import os
from io import StringIO  # Import StringIO from the io module
import time  # Import the time module
from urllib.parse import quote  # To URL encode SMILES
import plotly.express as px
import matplotlib.colors as mcolors

In [16]:
#functions to recover information and process data
def recover_LOTUS_data(q_code, output_folder):
    # Load the LOTUSDB CSV
    lotusdb_df = pd.read_csv(LOTUSDB, low_memory=False)
    
    # Filter LOTUSDB data for the specified Q code
    filtered_lotusdb = lotusdb_df[lotusdb_df['wikidata_Qcode'] == q_code]  # Ensure this column name matches your LOTUSDB CSV
    
    # Group and aggregate the data
    grouped_df = filtered_lotusdb.groupby("structure_inchikey").agg({
        # Add all the aggregation rules here
        # Example:
        "structure_wikidata": "first",
        "structure_inchi": "first",
        "structure_smiles": "first",
        "structure_molecular_formula": "first",
        "structure_exact_mass": "first",
        "structure_xlogp": "first",
        "structure_smiles_2D": "first",
        "structure_cid": "first",
        "structure_nameIupac": "first",
        "structure_nameTraditional": "first",
        "structure_taxonomy_npclassifier_01pathway": "first",
        "structure_taxonomy_npclassifier_02superclass": "first",
        "structure_taxonomy_npclassifier_03class": "first",
        "organism_wikidata": "first",
        "organism_taxonomy_gbifid": "first",
        "organism_taxonomy_ncbiid": "first",
        "organism_taxonomy_ottid": "first",
        "organism_taxonomy_01domain": "first",
        "organism_taxonomy_02kingdom": "first",
        "organism_taxonomy_03phylum": "first",
        "organism_taxonomy_04class": "first",
        "organism_taxonomy_05order": "first",
        "organism_taxonomy_06family": "first",
        "organism_taxonomy_07tribe": "first",
        "organism_taxonomy_08genus": "first",
        "organism_taxonomy_09species": "first",
        "organism_taxonomy_10varietas": "first",
        "reference_wikidata": lambda x: "|".join(map(str, x)),
        "reference_doi": lambda x: "|".join(map(str, x))
    }).reset_index()
    
    # Create 'chemical_superclass' and 'chemical_class' columns
    grouped_df['chemical_superclass'] = grouped_df['structure_taxonomy_npclassifier_01pathway'] + '-' + grouped_df['structure_taxonomy_npclassifier_02superclass']
    grouped_df['chemical_class'] = grouped_df['structure_taxonomy_npclassifier_01pathway'] + '-' + grouped_df['structure_taxonomy_npclassifier_03class']
    
    # Save the grouped data as a TSV file with the Q code as the filename
    output_filename = os.path.join(output_folder, f"{q_code}_reported.tsv")
    grouped_df.to_csv(output_filename, index=False, sep='\t')

    print(f"Saved grouped data for Q code {q_code} to {output_filename}")

def process_canopus_output(input_folder, output_folder, q_code, min_class_confidence=0.8):
    """
    Convert Canopus output to the format suitable for FBMN.

    Args:
    repository_path (str): Path to the Canopus output directory.
    pathout (str): Path to the output directory for the converted file.
    min_class_confidence (float, optional): Minimum class confidence cutoff. Defaults to 0.8.

    Returns:
    None
    """

    canopus_path = os.path.join(input_folder, "canopus_compound_summary.tsv")
    canopus_df = pd.read_csv(canopus_path, usecols=['id', 'molecularFormula', 'adduct', 'NPC#pathway',
                                                    'NPC#pathway Probability', 'NPC#superclass',
                                                    'NPC#superclass Probability', 'NPC#class',
                                                    'NPC#class Probability'], sep='\t')
    canopus_df.rename(columns={'NPC#class Probability': 'classProbability'}, inplace=True)
    canopus_df['shared name'] = canopus_df['id'].str.split('_').str[-1].astype(int)
    canopus_df.drop('id', axis=1, inplace=True)
    
    # Filter based on class probability
    canopus_df.drop(canopus_df[canopus_df.classProbability > min_class_confidence].index, inplace=True)
    
    # Drop unnecessary columns
    canopus_df.drop(['classProbability', 'NPC#superclass Probability', 'NPC#pathway Probability', ],
                    axis=1, inplace=True)
    #rename columns
    canopus_df.rename(columns={'shared name': 'row ID', 'adduct': 'adduct (canopus)',
                               'molecularFormula': 'MF (canopus)', 
                               'NPC#pathway':'structure_taxonomy_npclassifier_01pathway',
                               'NPC#superclass':'structure_taxonomy_npclassifier_02superclass',
                               'NPC#class':'structure_taxonomy_npclassifier_03class'}, inplace=True)

    # Create 'chemical_superclass' and 'chemical_class' columns
    canopus_df['chemical_superclass'] = canopus_df['structure_taxonomy_npclassifier_01pathway'] + '-' + canopus_df['structure_taxonomy_npclassifier_02superclass']
    canopus_df['chemical_class'] = canopus_df['structure_taxonomy_npclassifier_01pathway'] + '-' + canopus_df['structure_taxonomy_npclassifier_03class']
        
    # Write the converted data to a new file
    canopus_path_out = os.path.join(output_folder, q_code + "_annotated.tsv")
    canopus_df.to_csv(canopus_path_out, sep='\t', index=False)

    return None

def interpolate_color(color1, color2, factor: float):
    """Interpolate between two colors"""
    color1 = np.array(mcolors.to_rgb(color1))
    color2 = np.array(mcolors.to_rgb(color2))
    return mcolors.to_hex((1 - factor) * color1 + factor * color2)

def generate_shades(pathway, num_shades):
    # Define base colors for each Pathway
    pathway_shades= {
        'Terpenoids': ('#618264', '#D0E7D2'),  # Green start and lighter green end
        'Alkaloids': ('#305F72', '#5CBCE2'),   # Blue start and lighter blue end  
        'Shikimates and Phenylpropanoids': ('#80558C', '#CBA0AE'),  # Purple start and lighter purple end
        'Polyketides': ('#EF4B4B', '#EC8F6A'),  # Red start and lighter purple end
        'Fatty acids': ('#FF6C22', '#FF9209'),  # Orange start and lighter purple end
        'Amino acids and Peptides': ('#F4E869', '#FAF2D3'),  # Yellow start and lighter purple end
        'Carbohydrates': ('#65451F','#C8AE7D')  # Brown start and lighter purple end
    }
    base_color, end_color = pathway_shades.get(pathway, ('gray', 'lightgray'))

    if num_shades == 1:
        return [base_color]  # Return the base color if only one shade is requested
    shades = []
    for i in range(num_shades):
        factor = i / (num_shades - 1)
        shades.append(interpolate_color(base_color, end_color, factor))
    return shades

def split_chemical_superclass(row):
    # Check if the value is a string before splitting
    if isinstance(row['chemical_superclass'], str):
        parts = row['chemical_superclass'].split('-')
        if len(parts) == 2:
            return parts[0], parts[1]  # Pathway and Superclass are present
        else:
            return parts[0], 'Unknown'  # Only Pathway is present, or the format is not as expected
    else:
        # Return default values if the entry is NaN or not a string
        return 'Unknown', 'Unknown'
        
def barplot_sclass_species(output_folder, q_code, organims):
    """
    Process data from .tsv files in the output folder and visualize it with a stacked barplot.

    Parameters:
    output_folder (str): Path to the folder containing .tsv files.

    Returns:
    None
    """
    # Read reported data
    reported_data = pd.read_csv(os.path.join(output_folder, q_code+"_reported.tsv"), sep='\t')

    # Read annotated data
    annotated_data = pd.read_csv(os.path.join(output_folder, q_code+"_annotated.tsv"), sep='\t')

    # Combine reported and annotated data
    reported_data['Type'] = 'Reported'
    annotated_data['Type'] = 'Annotated'
    all_data = pd.concat([reported_data, annotated_data])

    # Rename the "organism_taxonomy_09species" column to "species"
    all_data.rename(columns={'organism_taxonomy_09species': 'species'}, inplace=True)

    # Remove 'API Error-API Error' and 'Not Classified-Not Classified'
    all_data = all_data[~all_data['chemical_superclass'].isin(['API Error-API Error', 'Not Classified-Not Classified'])]

    # Apply the function to each row
    all_data[['Pathway', 'superclass']] = all_data.apply(lambda row: split_chemical_superclass(row), axis=1, result_type='expand')

    # Process data for color mapping
    color_map = {}
    for pathway, superclasses in all_data.groupby('Pathway')['superclass'].unique().items():
        shades = generate_shades(pathway, len(superclasses))
        for superclass, shade in zip(superclasses, shades):
            color_map[f"{pathway}-{superclass}"] = shade

    # Group and aggregate data to calculate recurrence
    agg_data = all_data.groupby(['Type', 'chemical_superclass']).size().reset_index(name='recurrence')

    # Convert 'species' column to categorical data
    agg_data['Type'] = pd.Categorical(agg_data['Type'], categories=agg_data['Type'].unique(), ordered=True)

    # Get unique species names
    unique_species = agg_data['Type'].unique()

    # Get unique chemical superclasses and sort them alphabetically
    unique_superclasses = sorted(agg_data['chemical_superclass'].unique())

    # Calculate total recurrence for each species
    total_recurrence_per_species = agg_data.groupby('Type')['recurrence'].sum()

    # Create the stacked barplot with the custom color palette
    fig = px.bar(agg_data, y='Type', x='recurrence',
                 title='Stacked Barplot of reported and annotated Superclasses Occurrence for ' +organism,
                 labels={'recurrence': 'Recurrence'},
                 color='chemical_superclass',
                 color_discrete_map=color_map,
                 category_orders={'Type': unique_species, 'chemical_superclass': unique_superclasses},
                 orientation='h')

    # Modify the y-axis label
    #fig.update_yaxes(title_text='<i>Species<i>')
    
    # Set a white background
    fig.update_layout(plot_bgcolor='white')

    # Modify the size of the figure
    fig.update_layout(width=1500, height=1500)

    # Save the figure as an HTML file
    fig.write_html(os.path.join(output_folder, q_code+"_Wikidata_superclass_barplot_vs_annotations.html"))

    # Show the figure
    fig.show()

    return None

def barplot_pathway_species(output_folder, q_code, organims):
    """
    Process data from .tsv files in the output folder and visualize it with a stacked barplot.

    Parameters:
    output_folder (str): Path to the folder containing .tsv files.

    Returns:
    None
    """
    # Read reported data
    reported_data = pd.read_csv(os.path.join(output_folder, q_code+"_reported.tsv"), sep='\t')

    # Read annotated data
    annotated_data = pd.read_csv(os.path.join(output_folder, q_code+"_annotated.tsv"), sep='\t')

    # Combine reported and annotated data
    reported_data['Type'] = 'Reported'
    annotated_data['Type'] = 'Annotated'
    all_data = pd.concat([reported_data, annotated_data])

    # Rename the "organism_taxonomy_09species" column to "species"
    all_data.rename(columns={'organism_taxonomy_09species': 'species'}, inplace=True)
    all_data.rename(columns={'structure_taxonomy_npclassifier_01pathway': 'Pathway'}, inplace=True)

    # Remove 'API Error-API Error' and 'Not Classified-Not Classified'
    all_data = all_data[(all_data['Pathway'] != 'API Error') & (all_data['Pathway'] != 'Not Classified')]

    # Apply the function to each row
    all_data[['Pathway', 'superclass']] = all_data.apply(lambda row: split_chemical_superclass(row), axis=1, result_type='expand')

    # Define custom colors for the 7 pathway categories
    pathway_colors = {
        'Terpenoids': '#618264',  # Green start and lighter green end
        'Alkaloids': '#305F72',   # Blue start and lighter blue end
        'Shikimates and Phenylpropanoids': '#80558C',  # Purple start and lighter purple end
        'Polyketides': '#EF4B4B',  # Red start and lighter purple end
        'Fatty acids': '#FF6C22',  # Orange start and lighter purple end
        'Amino acids and Peptides': '#F4E869',  # Yellow start and lighter purple end
        'Carbohydrates': '#65451F' # Brown start and lighter purple end
    }

    # Group and aggregate data to calculate recurrence
    agg_data = all_data.groupby(['Type', 'Pathway']).size().reset_index(name='recurrence')

    # Convert 'species' column to categorical data
    agg_data['Type'] = pd.Categorical(agg_data['Type'], categories=agg_data['Type'].unique(), ordered=True)

    # Get unique species names
    unique_species = agg_data['Type'].unique()

    # Get unique chemical superclasses and sort them alphabetically
    unique_pathway = sorted(agg_data['Pathway'].unique())

    # Calculate total recurrence for each species
    total_recurrence_per_species = agg_data.groupby('Type')['recurrence'].sum()

    # Create the stacked barplot with the custom color palette
    fig = px.bar(agg_data, y='Type', x='recurrence',
                 title='Stacked Barplot of reported and annotated Superclasses Occurrence for ' +organism,
                 labels={'recurrence': 'Recurrence'},
                 color='Pathway',
                 color_discrete_map=pathway_colors,
                 category_orders={'Type': unique_species, 'Pathway': unique_pathway},
                 orientation='h')

    # Set a white background
    fig.update_layout(plot_bgcolor='white')

    # Modify the size of the figure
    fig.update_layout(width=1500, height=1500)

    # Save the figure as an HTML file
    fig.write_html(os.path.join(output_folder, q_code+"_Wikidata_pathway_barplot_vs_annotations.html"))

    # Show the figure
    fig.show()

    return None

Make sure you have everything indicated in the next cell

In [10]:
#before starting be sure you have the latest LOTUS frozen metadata in your compouter and set the path below.

LOTUSDB = 'C:/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv'#'/mnt/c/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv' #'C:/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv'#'/mnt/c/Users/quirosgu/Documents/Github/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv' #'/home/quirosgu/Desktop/FARMA-SHARE/RECHERCHE/FASIE_LAB/LuisQ/Yggdrasil/data_loc/LotusDB_inhouse_metadata.csv'

#Define the folder where you have your data : 

# Define the folder where you want to recover all processed data

input_folder = 'C:/Users/quirosgu/Desktop/Inmuno/Sirius_pos/' #put your path here

#you need to put in the root of this folder he 'canopus_compound_summary.tsv' obtained from SIRIUS

#do not modify from this line on

# Output folder for the processed CSV files
output_folder = f'{input_folder}output_data/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

Recover the reported chemistry of one species from Lotus

In [11]:
#1) recover the reported compounds in a species from Lotus 

#1.1 Look for the WikiData Qcode of the species here https://www.wikidata.org/wiki/Wikidata:Main_Page

q_code = 'Q55879736'#'Q1424919' #yourQCodegoeshere #example for the species Tripterygium wilfordii
organism = 'Maytenus woodsonii'#'Tripterygium wilfordii'

#1.2 Recover the reported chemistry from Lotus 

# Process the single Q code
recover_LOTUS_data(q_code, output_folder)


Saved grouped data for Q code Q55879736 to C:/Users/quirosgu/Desktop/Inmuno/Sirius_pos/output_data/Q55879736_reported.tsv


Process the data from Sirius of the samples we want to compare against the reported data

In [12]:
# just run the line, no need to do modifications. 

process_canopus_output(input_folder, output_folder, q_code, min_class_confidence=0.8)

Lets plot the reported chemistry agains the annotated one!

In [13]:
barplot_sclass_species(output_folder, q_code, organism)

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_32380\2943718948.py:188: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [15]:
barplot_pathway_species(output_folder, q_code, organism)

C:\Users\quirosgu\AppData\Local\Temp\ipykernel_32380\829308689.py:57: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

